<a href="https://colab.research.google.com/github/MasahiroAraki/MLCourse/blob/master/Python/answer/12a_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第12章 パターンマイニング

In [1]:
# colabで実行
!pip install -U mlxtend
!pip install scikit-surprise
import pkg_resources, imp
imp.reload(pkg_resources)

Requirement already up-to-date: mlxtend in /usr/local/lib/python3.7/dist-packages (0.18.0)


<module 'pkg_resources' from '/usr/local/lib/python3.7/dist-packages/pkg_resources/__init__.py'>

## 課題1

上記の規則抽出のコード例で求めた規則よりも、Lift値の高い規則を探すにはどうすればよいでしょうか。

In [2]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [3]:
!wget https://raw.githubusercontent.com/fracpete/wekamooc/master/dataminingwithweka/data/supermarket.arff

--2021-05-19 06:50:40--  https://raw.githubusercontent.com/fracpete/wekamooc/master/dataminingwithweka/data/supermarket.arff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2025871 (1.9M) [text/plain]
Saving to: ‘supermarket.arff.1’

supermarket.arff.1  100%[===================>]   1.93M  --.-KB/s    in 0.1s    

2021-05-19 06:50:40 (19.3 MB/s) - ‘supermarket.arff.1’ saved [2025871/2025871]



In [4]:
!sed -e "s/{ t}/{t}/" supermarket.arff > supermarket2.arff

In [5]:
from scipy.io import arff
data, meta = arff.loadarff('supermarket2.arff')
df = pd.DataFrame(data)
df2 = df.replace({b'?':False, b't':True})
df2 = df2.drop('total', axis=1)
df2

,department1,department2,department3,department4,department5,department6,department7,department8,department9,grocery misc,department11,baby needs,bread and cake,baking needs,coupons,juice-sat-cord-ms,tea,biscuits,canned fish-meat,canned fruit,canned vegetables,breakfast food,cigs-tobacco pkts,cigarette cartons,cleaners-polishers,coffee,sauces-gravy-pkle,confectionary,puddings-deserts,dishcloths-scour,deod-disinfectant,frozen foods,razor blades,fuels-garden aids,spices,jams-spreads,insecticides,pet foods,laundry needs,party snack foods,...,department177,department178,department179,casks white wine,casks red wine,750ml white nz,750ml red nz,750ml white imp,750ml red imp,sparkling nz,sparkling imp,brew kits/accesry,department189,port and sherry,ctrled label wine,department192,department193,department194,department195,department196,department197,department198,department199,non host support,department201,department202,department203,department204,department205,department206,department207,department208,department209,department210,department211,department212,department213,department214,department215,department216
0,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,True,False,True,False,False,True,False,False,False,True,True,True,True,False,True,False,True,True,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,True,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,False,True,True,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,True,False,False,False,False,False,False,False,True,True,True,False,False,False,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,True,True,False,False,False,True,False,True,False,True,Fa

FPGrowthでmin_supportを下げて多くの頻出項目集合を求め、association_rulesで規則を評価します。ただしmin_supportをあまり下げすぎると、あまり意味のない項目集合が出てきます。

In [6]:
freq2 = fpgrowth(df2, min_support=0.2, use_colnames=True)

In [7]:
association_rules(freq2, metric="lift", min_threshold=1.5)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(bread and cake, biscuits, vegetables)","(frozen foods, fruit)",0.321375,0.402204,0.200778,0.624748,1.553309,0.071520,1.593050
1,"(frozen foods, bread and cake, vegetables)","(fruit, biscuits)",0.334558,0.397018,0.200778,0.600129,1.511594,0.067953,1.507943
2,"(frozen foods, bread and cake, fruit)","(biscuits, vegetables)",0.334558,0.381241,0.200778,0.600129,1.574148,0.073231,1.547398
3,"(biscuits, vegetables)","(frozen foods, bread and cake, fruit)",0.381241,0.334558,0.200778,0.526644,1.574148,0.073231,1.405796
4,"(fruit, biscuits)","(frozen foods, bread and cake, vegetables)",0.397018,0.334558,0.200778,0.505716,1.511594,0.067953,1.346274
5,"(frozen foods, fruit)","(bread and cake, biscuits, vegetables)",0.402204,0.321375,0.200778,0.499194,1.553309,0.071520,1.355067


## 課題2

教科書p.226の演習問題12-3の手順に従い、MovieLensデータセットに対してscikit-surpriseを用いてNMFを行ってください。余力があれば、ハイパーパラメータを変更して、性能に与える影響を観察してください。

In [8]:
from surprise import NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [9]:
# movielens-100k データを読み込みます。読み込みに承諾(Yと入力)が必要です。
X = Dataset.load_builtin('ml-100k')

In [10]:
# 5-fold CVで、平均二乗誤差と平均絶対誤差を表示します。
cross_validate(NMF(), X, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9519  0.9607  0.9684  0.9699  0.9698  0.9641  0.0070  
MAE (testset)     0.7486  0.7554  0.7618  0.7608  0.7620  0.7577  0.0052  
Fit time          5.75    5.89    5.67    5.74    5.69    5.75    0.08    
Test time         0.29    0.14    0.22    0.18    0.19    0.20    0.05    


{'fit_time': (5.7525269985198975,
  5.886185169219971,
  5.674530744552612,
  5.74294638633728,
  5.686476945877075),
 'test_mae': array([0.74859091, 0.7554082 , 0.76179869, 0.76081876, 0.76198282]),
 'test_rmse': array([0.95188297, 0.96072346, 0.96836822, 0.96987514, 0.96980776]),
 'test_time': (0.2931842803955078,
  0.13692879676818848,
  0.21803045272827148,
  0.1838676929473877,
  0.1888124942779541)}

scikit-surprise NMFのデフォルトの圧縮次元数(n_factors)は15です。これを下げると、実行時間は早くなっていますが、性能は悪くなっています。

In [11]:
cross_validate(NMF(n_factors=5), X, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0496  1.0687  1.0545  1.0688  1.0569  1.0597  0.0077  
MAE (testset)     0.8616  0.8771  0.8648  0.8791  0.8714  0.8708  0.0068  
Fit time          4.36    4.32    4.33    4.28    4.30    4.32    0.03    
Test time         0.27    0.14    0.13    0.14    0.25    0.18    0.06    


{'fit_time': (4.364952564239502,
  4.315894603729248,
  4.332253932952881,
  4.277153015136719,
  4.300522327423096),
 'test_mae': array([0.86161031, 0.87709676, 0.86479302, 0.87913304, 0.87143492]),
 'test_rmse': array([1.04962915, 1.06870176, 1.05454012, 1.06877009, 1.05692208]),
 'test_time': (0.26619625091552734,
  0.14427423477172852,
  0.12857747077941895,
  0.13547992706298828,
  0.2462613582611084)}